In [1]:
import pandas as pd
from google.colab import files
dataset=files.upload()


Saving INR-vs-USD.csv to INR-vs-USD.csv


In [38]:
import numpy as np
dataset=pd.read_csv('INR-vs-USD.csv')

# Setting the date as the index
df = dataset.copy()
df['Date'] = pd.to_datetime(df['Date'])
df.index = df['Date']
del df['Date']

#print(df.head(10))

# Interpolation
df_interpol = df.resample('D').mean()
df_interpol['INR vs USD'] = df_interpol['INR vs USD'].interpolate()

# Resetting index as [0,1...]
k=df_interpol.index
df_interpol.index = np.arange(0, len(df_interpol))
df_interpol = df_interpol.assign(Date = k) 
print(df_interpol)

       INR vs USD       Date
0        8.020000 1973-01-02
1        8.020000 1973-01-03
2        8.000000 1973-01-04
3        8.010000 1973-01-05
4        8.006667 1973-01-06
...           ...        ...
16302   64.100000 2017-08-21
16303   64.070000 2017-08-22
16304   64.040000 2017-08-23
16305   64.040000 2017-08-24
16306   64.000000 2017-08-25

[16307 rows x 2 columns]


In [0]:
#Splitting into train and test
train_size = int(len(df_interpol) * 0.80)
train,test=df_interpol[:].iloc[:train_size],df_interpol[:].iloc[train_size:]

# create dataset 
def create_dataset(dataset, n_steps=1):
    X, Y = [], []
    for i in range(len(dataset)-n_steps-1):
        
        a=dataset['INR vs USD'].iloc[i:i+n_steps]
        X.append(a)
       
        Y.append( dataset['INR vs USD'].iloc[i+n_steps])
    return np.array(X), np.array(Y)
    
n_steps = 2
#print(df_interpol['INR vs USD'].iloc[0:30])

x_train, y_train = create_dataset(train, n_steps)
x_test, y_test = create_dataset(test, n_steps)

In [130]:
print(x_train.shape)
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))


(13042, 2)


In [131]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout


model = Sequential()
model.add(LSTM(100, input_shape=(1,n_steps)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(x_train, y_train, epochs=20, batch_size=70, validation_data=(x_test, y_test), verbose=1)



Train on 13042 samples, validate on 3259 samples
Epoch 1/20
13042/13042 [==============================] - 4s 301us/step - loss: 484.0859 - val_loss: 1534.5465
Epoch 2/20
13042/13042 [==============================] - 1s 113us/step - loss: 169.3711 - val_loss: 906.0594
Epoch 3/20
13042/13042 [==============================] - 1s 111us/step - loss: 66.4907 - val_loss: 574.4973
Epoch 4/20
13042/13042 [==============================] - 1s 111us/step - loss: 28.3663 - val_loss: 401.3950
Epoch 5/20
13042/13042 [==============================] - 1s 111us/step - loss: 13.2350 - val_loss: 304.0512
Epoch 6/20
13042/13042 [==============================] - 1s 112us/step - loss: 7.3504 - val_loss: 248.8570
Epoch 7/20
13042/13042 [==============================] - 1s 111us/step - loss: 5.0198 - val_loss: 215.4258
Epoch 8/20
13042/13042 [==============================] - 1s 111us/step - loss: 3.9722 - val_loss: 194.0189
Epoch 9/20
13042/13042 [==============================] - 1s 109us/step - loss:

In [132]:
from sklearn.metrics import mean_squared_error
y_pred=model.predict(x_test)
y_test=np.reshape(y_test,(y_test.shape[0],1))
print('Test Root Mean Squared Error:',np.sqrt(mean_squared_error(y_test[0], y_pred[0])))

print(y_pred)
print(y_test)


Test Root Mean Squared Error: 0.4754035949707003
[[44.824596]
 [44.704624]
 [44.817944]
 ...
 [48.839565]
 [48.839043]
 [48.838284]]
[[45.3 ]
 [45.65]
 [45.92]
 ...
 [64.07]
 [64.04]
 [64.04]]


In [0]:
#number of steps=5 mse=0.971301
#number of step=30 mse=5.83
#number of steps=2 mse=0.475